# Дообучение большой языковой модели (LLM)

Ноутбук можно запустить на платформе ML Space, [создав Jupyter Server](https://cloud.ru/docs/aicloud/mlspace/concepts/guides/guides__jupyter/environments__environments__jupyter-server__create-new-jupyter-server.html).

## 1. Установка зависимостей и импорт библиотек

Перед началом работы установим необходимые пакеты для обработки данных и обучения моделей.

In [1]:
!pip install -Uqqq pip
!pip install -qqq bitsandbytes torch transformers peft \
    accelerate datasets loralib==0.1.1 einops==0.6.1 scipy sentencepiece

In [2]:
import os
import torch
import transformers
from datasets import load_dataset
from peft import (
    LoraConfig,
    PeftConfig,
    PeftModel,
    get_peft_model,
    prepare_model_for_kbit_training
)
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig
)

import mlflow


os.environ["CUDA_VISIBLE_DEVICES"] = "0"

## 2. Загрузка базовой модели

Загрузим подходящую предварительно обученную модель. В этом примере выбрана "Intel/neural-chat-7b-v3-1". Можно выбрать другую, заменив значение переменной `MODEL_NAME`.

Используются следующие параметры:

- `load_in_4bit` — загрузка модели в 4-битном формате с уменьшением ее размера в памяти;

- `bnb_4bit_use_double_quant` — двойная квантизация для дополнительного уменьшения размера модели;

- `bnb_4bit_quant_type` — тип квантизации ("nf4"), который определяет, как модель будет сжиматься;

- `bnb_4bit_compute_dtype` — тип данных для вычислений (torch.bfloat16), который позволяет уменьшить потребление памяти и ускорить вычисления, сохраняя при этом точность.

Сжимаем модель с помощью библиотеки `BitsAndBytes`.

In [3]:
MODEL_NAME = "Intel/neural-chat-7b-v3-1"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [4]:
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    trust_remote_code=True,
    quantization_config=bnb_config
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

## 3. Добавление LoRA

Реализуется метод LoRa, который позволяет изменять только небольшую часть параметров модели.

In [5]:
def print_trainable_parameters(model):
  """
  Calculates and displays the total number of model parameters and the number of trainable parameters.
  """
  trainable_params = 0
  all_param = 0
  for _, param in model.named_parameters():
    all_param += param.numel()
    if param.requires_grad:
      trainable_params += param.numel()
  print(
      f"trainable params: {trainable_params} || all params: {all_param} || trainables%: {100 * trainable_params / all_param}"
  )

In [6]:
model.gradient_checkpointing_enable()

In [7]:
model = prepare_model_for_kbit_training(model)

Создадим конфигурация для LoRA (Low-Rank Adaptation), с помощью которой можно адаптировать только небольшую часть весов модели. 

Конфигурация имеет следующие параметры:

- `r` и `lora_alpha` — параметры, контролирующие размер и мощность адаптации;

- `target_modules` — список модулей модели, к которым будет применена адаптация LoRA;

- `lora_dropout` — применение dropout к адаптированным весам;

- `bias` — настройка использования смещения в адаптации;

- `task_type` — тип задачи, для которой настраивается модель, в нашем случае это генерация текста.



In [8]:
config = LoraConfig(
    r=8,
    lora_alpha=32,
    #target_modules=["query_key_value"],
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)

In [9]:
print_trainable_parameters(model)

trainable params: 6815744 || all params: 3758886912 || trainables%: 0.18132346515244138


## 4. Дообучение модели

Проверим, как модель генерирует ответы на основе заданного запроса.


In [10]:
prompt = """
<human>: midjourney prompt for a girl sit on the mountain
<assistant>:
""".strip()

Зададим параметры для генерации текста:
- `max_new_tokens` — максимальное количество новых токенов, которые модель может сгенерировать;

- `temperature` — степень случайности в выборе слов, где меньшее значение приводит к более предсказуемому тексту;

- `top_p` — вероятностный порог выбора слов; рассматриваться будут только слова с вероятностью выше этого порога;

- `num_return_sequences` — количество возвращаемых последовательностей;

- `pad_token_id` и `eos_token_id` — идентификаторы токенов заполнения и окончания предложения.

In [11]:
generation_config = model.generation_config
generation_config.max_new_tokens = 200
generation_config.temperature = 0.7
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

Преобразуем запрос в формат, пригодный для модели, с помощью токенизатора и переместим данные на GPU.
`return_tensors="pt"` указывает на то, что возвращаемые тензоры должны быть в формате PyTorch.

In [12]:
device = "cuda:0"

encoding = tokenizer(prompt, return_tensors="pt").to(device)

In [13]:
%%time
with torch.inference_mode():
  outputs = model.generate(
      input_ids = encoding.input_ids,
      attention_mask = encoding.attention_mask,
      generation_config = generation_config
  )

/home/jovyan/.mlspace/envs/llm_trainer/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:540: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/jovyan/.mlspace/envs/llm_trainer/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:545: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


CPU times: user 3.82 s, sys: 232 ms, total: 4.06 s
Wall time: 4.21 s


In [14]:
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

<human>: midjourney prompt for a girl sit on the mountain
<assistant>: A young girl, dressed in a warm, cozy outfit, sits on a large boulder overlooking a vast, snow-capped mountain range. The sun is setting behind her, casting a golden glow on her face and the surrounding landscape. She gazes into the distance, lost in thought, as the cool breeze gently ruffles her hair.


## 5. Загрузка набора данных

Текстовый запрос к модели генерируется, обрабатывается и передается в модель. 
Для обучения использован датасет [Mid Journey Prompts от Hugging Face](https://huggingface.co/datasets/bittu9988/mid_journey_prompts).

In [15]:
data = load_dataset("bittu9988/mid_journey_prompts")

In [17]:
def generate_prompt(data_point):
  return f"""
<human>: {data_point["User"]}
<assistant>: {data_point["Prompt"]}
""".strip()

def generate_and_tokenize_prompt(data_point):
  full_prompt = generate_prompt(data_point)
  tokenized_full_prompt = tokenizer(full_prompt, padding=True, truncation=True)
  return tokenized_full_prompt

In [18]:
data = data["train"].shuffle().map(generate_and_tokenize_prompt)

Map:   0%|          | 0/289 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


## 6. Обучение

Задаются параметры модели, и начинается ее дообучение.

Зададим переменные окружения для использования MLflow. Подробнее — в [документации Hugging Face](https://huggingface.co/docs/transformers/v4.43.3/en/main_classes/callback#transformers.integrations.MLflowCallback).

In [19]:
os.environ["MLFLOW_EXPERIMENT_NAME"] = "trainer-llm"
os.environ["MLFLOW_TRACKING_URI"] = "/home/jovyan/mlruns"
os.environ["MLFLOW_FLATTEN_PARAMS"] = "1"
os.environ["HF_MLFLOW_LOG_ARTIFACTS"]="1"

Создадим объект `TrainingArguments`, который содержит различные параметры для обучения:

- `per_device_train_batch_size` — размер батча обучения на каждом устройстве;

- `gradient_accumulation_steps` — количество шагов накопления градиента перед их обратным распространением;

- `num_train_epochs` — количество эпох обучения;

- `learning_rate` — скорость обучения;

- `fp16` — использование 16-битной точности с плавающей запятой для ускорения обучения и снижения потребления памяти;

- `save_total_limit` — максимальное количество сохраняемых чекпоинтов;

- `logging_steps` — частота логирования;

- `output_dir` — директория для сохранения результатов обучения;

- `optim` — оптимизатор, здесь используется 8-битная версия AdamW;

- `lr_scheduler_type` — тип планировщика скорости обучения;

- `warmup_ratio` — доля общего числа шагов обучения, в течение которых скорость обучения линейно увеличивается до заданной.

In [20]:
training_args = transformers.TrainingArguments(
      per_device_train_batch_size=1,
      gradient_accumulation_steps=8,
      num_train_epochs=4,
      learning_rate=2e-4,
      fp16=True,
      save_total_limit=3,
      logging_steps=1,
      output_dir="experiments",
      optim="paged_adamw_8bit",
      lr_scheduler_type="cosine",
      warmup_ratio=0.05,
)

`Trainer` отвечает за процесс обучения модели и имеет следующие параметры:

- `model` — модель, которая будет обучаться;

- `train_dataset` — набор данных для обучения;

- `args` — аргументы обучения, определенные выше;

- `data_collator` — объект, который формирует батчи из данных.

In [21]:
trainer = transformers.Trainer(
    model=model,
    train_dataset=data,
    args=training_args,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)

Эта настройка отключает кеширование в модели, что может быть полезно для экономии памяти во время обучения.

In [22]:
model.config.use_cache = False

Запустим процесс обучения с помощью метода `train()`.

In [23]:
trainer.train()

2024/07/31 17:45:54 INFO mlflow.tracking.fluent: Experiment with name 'trainer-llm' does not exist. Creating a new experiment.
/home/jovyan/.mlspace/envs/llm_trainer/lib/python3.11/site-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
1,4.297500
2,4.015500
3,4.256600
4,4.157600
5,3.928300
6,3.659200
7,3.311100
8,3.147700
9,3.203400
10,2.776600


/home/jovyan/.mlspace/envs/llm_trainer/lib/python3.11/site-packages/_distutils_hack/__init__.py:32: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition will fail. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(
2024/07/31 17:50:52 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.


TrainOutput(global_step=144, training_loss=1.2027036458667781, metrics={'train_runtime': 298.6398, 'train_samples_per_second': 3.871, 'train_steps_per_second': 0.482, 'total_flos': 3939821811425280.0, 'train_loss': 1.2027036458667781, 'epoch': 3.986159169550173})

In [ ]:
mlflow.end_run() 

## 7. Сохранение модели и инференс

Сохраним в папку `trained-model` текущее состояние модели.

In [25]:
model.save_pretrained("trained-model")

Загрузим конфигурацию PEFT (Parameter-Efficient Fine-Tuning) из сохраненной модели. Далее создадим новый экземпляр модели с помощью этой конфигурации. 

Параметры `return_dict=True`, `quantization_config=bnb_config`, `device_map="auto"` и `trust_remote_code=True` настраивают поведение модели, включая формат возвращаемых данных, настройки квантизации, автоматическое распределение по устройствам и доверие к исполняемому коду.



In [26]:
config = PeftConfig.from_pretrained('./trained-model')
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    return_dict=True,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

Loading checkpoint shards: 100%|██████████| 2/2 [00:03<00:00,  1.58s/it]


In [27]:
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)
tokenizer.pad_token = tokenizer.eos_token

In [28]:
model = PeftModel.from_pretrained(model, './trained-model')

In [29]:
generation_config = model.generation_config
generation_config.max_new_tokens = 200
generation_config.temperature = 0.7
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

Создадим и токенизируем тестовый запрос.

In [30]:
device = "cuda:0"

prompt = """
<human>: midjourney prompt for 
<assistant>:
""".strip()

encoding = tokenizer(prompt, return_tensors="pt").to(device)

В блоке ``with torch.no_grad()`` отключается вычисление градиентов для экономии ресурсов. 

Метод model.generate используется для генерации текста на основе входных данных (`input_ids`, `attention_mask`) и предварительно настроенных параметров генерации (`generation_config`).

In [31]:
%%time

with torch.no_grad():
  model.config.use_cache = False
  outputs = model.generate(
      input_ids = encoding.input_ids,
      attention_mask = encoding.attention_mask,
      generation_config = generation_config
  )

/home/user/conda/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:540: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/user/conda/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:545: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


CPU times: user 10.6 s, sys: 68 ms, total: 10.6 s
Wall time: 10.6 s


Выведем сгенерированный текст.

In [32]:
print(tokenizer.decode(outputs[0]))

<s> <human>: midjourney prompt for 
<assistant>: 8k octane render of a realistic furry creature in a forest environment with Pixar render, pencil test, pixar style, character studio, character render, high detail, hyper-realistic, photorealistic, cinematic, 8k, octane render, environment, nature, forest, greens, plants, high detail, 8k, octane render, cinematic, 8k, high detail, hyper-realistic, photorealistic, pixar style, character studio, character render, environment, nature, forest, greens, plants, high detail, 8k, octane render, cinematic, 8k, high detail, hyper-realistic, photorealistic, pixar style, character studio, character render, environment, nature, forest, greens, plants, high detail, 8k, octane render, cinematic, 8k,
